Project: MNIST Digit Classification
File Name: mnist_comparison.ipynb
Author: Emily Au

In [ ]:
# Dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

print(f"PyTorch: {torch.__version__}")
print(f"TensorFlow: {tf.__version__}")

In [ ]:
# Data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

# DataLoaders (Windows fix)
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Model
class MNISTNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model_pt = MNISTNet()
criterion = nn.CrossEntropyLoss()
optimizer_pt = optim.Adam(model_pt.parameters(), lr=1e-3)

# Train 5 epochs
EPOCHS = 5
model_pt.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for x_batch, y_batch in train_loader:
        optimizer_pt.zero_grad()
        logits = model_pt(x_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer_pt.step()
        total_loss += loss.item()
    print(f'PyTorch Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')

# Test
model_pt.eval()
correct, total = 0, 0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        pred = model_pt(x_batch).argmax(1)
        correct += (pred == y_batch).sum().item()
        total += y_batch.size(0)
pt_acc = 100 * correct / total
print(f'PyTorch: {pt_acc:.2f}%')

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype('float32')/255., x_test.astype('float32')/255.
x_train, x_test = x_train.reshape(60000,784), x_test.reshape(10000,784)

BATCH_SIZE = 64
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60000).batch(BATCH_SIZE)
val_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

model_tf = keras.Sequential([layers.Dense(128, activation='relu', input_shape=(784,)), layers.Dense(10)])
model_tf.compile(optimizer=keras.optimizers.Adam(1e-3), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model_tf.fit(train_ds, epochs=5, validation_data=val_ds, verbose=1)

_, tf_acc = model_tf.evaluate(val_ds, verbose=0)
print(f'TensorFlow: {tf_acc*100:.2f}%')

In [ ]:
pd.DataFrame({
    'Framework': ['PyTorch', 'TensorFlow'],
    'Lines': [89, 58],
    'Style': ['Imperative', 'Declarative'],
    'Accuracy': [f'{pt_acc:.2f}%', f'{tf_acc*100:.2f}%']
})

In [ ]:
torch.save(model_pt.state_dict(), 'mnist_pytorch.pt')
model_tf.save('mnist_tf.keras')
print('Models saved!')
print('- PyTorch: mnist_pytorch.pt')
print('- TensorFlow: mnist_tf.keras')